# Installing Dependencies

In [1]:
import numpy as np
import cv2
import face_recognition
import os

# Reading Dataset Image

In [3]:
def read_img(path):
   img = cv2.imread(path)
   (h, w) = img.shape[:2]
   width = 400
   ratio = width / float(w)
   height = int(h * ratio)
   return cv2.resize(img, (width, height))

dataset_encodings = []
dataset_names = []
face_enc = []
dataset_dir = 'dataset'

for file in os.listdir(dataset_dir):
    img = read_img(dataset_dir + '/' + file)
    img_enc = face_recognition.face_encodings(img)[0]
    if len(img_enc) == 0:
        print("WARNING: No faces found in {}. Ignoring file.".format(file))
    else:
        dataset_encodings.append(img_enc)
        dataset_names.append(file.split('.')[0])

# Webcam

In [4]:
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    rgb_frame = frame[:, :, ::-1]

    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(dataset_encodings, face_encoding)

        name = "Unknown"

        if True in matches:
             first_match_index = matches.index(True)
             name = dataset_names[first_match_index]
            
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()